# 소셜 데이터 분석을 위한 크롤링

비, 흐림, 안개와 같은 좋지 않은 날씨의 해시태그들과 부정적인 감정을 표현한 해시태그가 많이 사용되는지 알아보기 위함. 상관관계라도 찾을 수 있으면 더할 나위 없이 좋겠다.

### 첫 번째 시도
참고: [링크 텍스트](https://mokeya.tistory.com/68)

In [2]:
pip install selenium

     |████████████████████████████████| 911kB 6.7MB/s 


In [3]:
#필요 패키지 호출 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time

In [4]:
#함수 정의: 검색어 조건에 따른 url 생성
# 내가 원하는 키워드를 받아서 해당 키워드 검색 결과 페이지의 url을 생성하는 함수를 정의

def insta_searching(word):
  url="https://www.instagram.com/explore/tags/" +str(word)
  return url

In [5]:
#함수 정의: 열린 페이지에서 첫 번째 게시물 클릭 + sleep 메소드 통해 시차 두기
def select_firts(driver):
  first=driver.find_elements_by_css_selector("div._9AhH0")[0]
  first.click()
  time.sleep(3)

In [6]:
#함수 정의: 본문 내용, 작성일자, 좋아요 수, 위치정보, 해시태그 가져오기
def get_content(driver):
  html=driver.page_source
  soup=BeautifulSoup(html, 'lxml')
  #본문 내용
  try:
    content=soup.select('div.C4VMK>span')[0].text
  except:
    content = ' '
  # 해시태그
  tags = re.findall(r'#[^\s#,\\]+', content)
  # 작성일자
  date=soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
  # 좋아요
  try:
    like=soup.select('div.Nm9Fw > button')[0].text[4:-1]
  except:
    like = 0
  #위치
  try:
    place=soup.select('div.M30cS')[0].text
  except:
    place=''
  
  data=[content, date, like, place, tags]
  return data

In [7]:
#함수 정의: 첫 게시물 클릭 후 다음 게시물 클릭

def move_next(driver):
  right=driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
  right.click()
  time.sleep(3)


In [ ]:
#크롤링 시작
# driver.get(url)을 통해 검색 페이지 접속하고, target 변수에 크롤링할 게시글의 수를 바인딩

driver=